In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
train_ds, val_ds = keras.utils.text_dataset_from_directory(
    './aclImdb/train/',
    validation_split= 0.2,
    subset='both',
    seed=225,
)

test_ds = keras.utils.text_dataset_from_directory(
    './aclImdb/test/',
    seed=225,
)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


In [3]:
max_tokens = 25000
max_length = 600

text_vectorizer = keras.layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode='int',
    output_sequence_length=max_length,
)

text_only_train_ds = train_ds.map(lambda x, y: x)

text_vectorizer.adapt(text_only_train_ds)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [4]:
int_train_ds = train_ds.map(lambda x, y: (text_vectorizer(x), y), num_parallel_calls=tf.data.AUTOTUNE)
int_val_ds = val_ds.map(lambda x, y: (text_vectorizer(x), y), num_parallel_calls=tf.data.AUTOTUNE)
int_test_ds = test_ds.map(lambda x, y: (text_vectorizer(x), y), num_parallel_calls=tf.data.AUTOTUNE)

In [5]:
inputs = keras.Input(shape=(None,), dtype='int64')

embeded = keras.layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)

x = keras.layers.Bidirectional(keras.layers.LSTM(32, return_sequences=True))(embeded)
x = keras.layers.Bidirectional(keras.layers.LSTM(32))(x)
x = keras.layers.Dropout(0.5)(x)

outputs = keras.layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs, outputs)
model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['accuracy'],
)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         6400000   
                                                                 
 bidirectional (Bidirectiona  (None, None, 64)         73984     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               24832     
 nal)                                                            
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65    

In [7]:
model.fit(int_train_ds, validation_data=int_val_ds, batch_size=64, epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('checkpoints/bilstm.h5', save_weights_only=True)])


Epoch 1/10
625/625 [==============================] - 101s 150ms/step - loss: 0.5209 - accuracy: 0.7456 - val_loss: 0.5766 - val_accuracy: 0.7966
Epoch 2/10
625/625 [==============================] - 64s 103ms/step - loss: 0.3661 - accuracy: 0.8608 - val_loss: 0.3485 - val_accuracy: 0.8570
Epoch 3/10
625/625 [==============================] - 51s 81ms/step - loss: 0.2842 - accuracy: 0.8950 - val_loss: 0.3172 - val_accuracy: 0.8834
Epoch 4/10
625/625 [==============================] - 48s 77ms/step - loss: 0.2405 - accuracy: 0.9147 - val_loss: 0.3223 - val_accuracy: 0.8684
Epoch 5/10
625/625 [==============================] - 47s 75ms/step - loss: 0.2003 - accuracy: 0.9302 - val_loss: 0.3522 - val_accuracy: 0.8788
Epoch 6/10
625/625 [==============================] - 45s 73ms/step - loss: 0.1710 - accuracy: 0.9416 - val_loss: 0.5572 - val_accuracy: 0.8328
Epoch 7/10
625/625 [==============================] - 44s 71ms/step - loss: 0.1454 - accuracy: 0.9505 - val_loss: 0.3677 - val_accura

In [6]:
model.fit(int_train_ds, validation_data=int_val_ds, batch_size=64, epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('checkpoints/2-layer-bilstm.h5', save_weights_only=True)])


Epoch 1/10
625/625 [==============================] - 143s 212ms/step - loss: 0.5059 - accuracy: 0.7556 - val_loss: 0.3417 - val_accuracy: 0.8630
Epoch 2/10
625/625 [==============================] - 102s 163ms/step - loss: 0.3321 - accuracy: 0.8743 - val_loss: 0.3866 - val_accuracy: 0.8598
Epoch 3/10
625/625 [==============================] - 91s 145ms/step - loss: 0.2641 - accuracy: 0.9063 - val_loss: 0.4132 - val_accuracy: 0.8524
Epoch 4/10
625/625 [==============================] - 87s 139ms/step - loss: 0.2117 - accuracy: 0.9257 - val_loss: 0.4294 - val_accuracy: 0.8448
Epoch 5/10
625/625 [==============================] - 86s 138ms/step - loss: 0.1683 - accuracy: 0.9422 - val_loss: 0.4247 - val_accuracy: 0.8754
Epoch 6/10
625/625 [==============================] - 84s 134ms/step - loss: 0.1430 - accuracy: 0.9538 - val_loss: 0.5524 - val_accuracy: 0.8460
Epoch 7/10
625/625 [==============================] - 82s 132ms/step - loss: 0.1125 - accuracy: 0.9656 - val_loss: 0.4644 - val_